In [1]:
import sys
sys.path.append('/home/lumargot/trachoma/src/py')

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # put -1 to not use any

In [2]:
import os
import pandas as pd
import numpy as np 

import torch
import cv2
torch.set_float32_matmul_precision('medium')

from loaders.tt_dataset import TTDataModulePatch, TrainTransformsFullSeg, EvalTransformsFullSeg

from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import SimpleITK as sitk

from matplotlib.patches import Rectangle

from  PIL  import  Image
import math
import numpy as np 


import torch
from torch import Tensor, nn
import torch.nn.functional as F

from torchvision import models
from torchvision import transforms
from torchvision import ops
import torchmetrics
from PIL import Image
import monai

import lightning.pytorch as pl


/home/lumargot/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
[neptune] [warning] NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.


In [3]:
mount_point = "/CMF/data/lumargot/trachoma/B images one eye/"

df = pd.read_csv('/CMF/data/lumargot/trachoma/patches/csv/new/Pret_excluded_clean.csv')

df_test = pd.read_csv('/CMF/data/lumargot/trachoma/patches/csv/new/Pret_excluded_clean_fold0_test.csv')
df_val = pd.read_csv('/CMF/data/lumargot/trachoma/patches/csv/new/Pret_excluded_clean_fold0_train_test.csv')    
df_train = pd.read_csv('/CMF/data/lumargot/trachoma/patches/csv/new/Pret_excluded_clean_fold0_train_train.csv')


In [4]:
df_train['class'].unique()

array([0, 1, 4, 2, 3, 5, 6])

In [5]:
df = df.loc[~df['label'].isin(['Healthy', 'Reject'])]

print(df[['label', 'class']].drop_duplicates())
concat_eca = ['ECA', 'Gap','overcorrection']
concat_ptt = ['Entropion','Short Incision']

eca_val = df.loc[ df['label'] == concat_eca[0]]['class'].unique()
ptt_val = df.loc[ df['label'] == concat_ptt[0]]['class'].unique()


df.loc[ df['label'].isin(concat_eca), "class" ] = eca_val[0]
df.loc[ df['label'].isin(concat_ptt), "class" ] = ptt_val[0]

unique_classes = sorted(df['class'].unique())
class_mapping = {value: idx for idx, value in enumerate(unique_classes)}

             label  class
12             ECA      1
16       Entropion      2
19             Gap      3
24  overcorrection      4
70  Short Incision      5


In [5]:
conflict = 0
subjects_id = df['filename'].unique()
for subject in subjects_id:

  df_sub = df.loc[ df['filename'] == subject]

  df_labels = df_sub['class'].unique()

  if len(df_labels)> 1:
    conflict += 1
    # print(df_sub['class'].value_counts(normalize=True))

ratio = 100* conflict/len(subjects_id)

total number of conflict = 137 -> 13.7% of dataset

with a distribution between [50, 60[ = 39           -> 3.9%
with a distribution between [60, 70[ = 44 + 39 = 85 -> 8.5%

In [ ]:
class TimeDistributed(nn.Module):
    def __init__(self, module):
        super(TimeDistributed, self).__init__()
        self.module = module
 
    def forward(self, input_seq):
        assert len(input_seq.size()) > 2
 
        # reshape input data --> (samples * timesteps, input_size)
        # squash timesteps

        size = input_seq.size()

        batch_size = size[0]
        time_steps = size[1]

        size_reshape = [batch_size*time_steps] + list(size[2:])
        reshaped_input = input_seq.contiguous().view(size_reshape)
 
        output = self.module(reshaped_input)
        
        output_size = output.size()
        output_size = [batch_size, time_steps] + list(output_size[1:])
        output = output.contiguous().view(output_size)

        return output


class EfficientNetV2SYOLTPatchv2(pl.LightningModule):
    def __init__(self, num_classes,**kwargs):
        super(EfficientNetV2SYOLTPatchv2, self).__init__()        
        
        self.save_hyperparameters()
        self.num_classes = num_classes

        class_weights = None
        if hasattr(self.hparams, "class_weights"):
            class_weights = torch.tensor(self.hparams.class_weights).to(torch.float32)
            
        self.loss = nn.CrossEntropyLoss(weight=class_weights)
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=self.num_classes)


        model_feat = nn.Sequential(
            models.efficientnet_v2_s(pretrained=True).features,
            ops.Conv2dNormActivation(1280, 1536),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(start_dim=1)
            )
        self.F = TimeDistributed(model_feat)
        
        self.V = nn.Linear(in_features=1536, out_features=64)

        #####  multihead attention ####
        self.A = nn.MultiheadAttention(embed_dim=1024, num_heads=4, batch_first=True)
        self.V2A = nn.Linear(64, 1024)
        
        self.P = nn.Linear(in_features=1024, out_features=self.num_classes)        


        # self.train_transform = transforms.Compose(
        #     [
        #         RandomRotate(degrees=90, keys=["img", "seg"], interpolation=[transforms.functional.InterpolationMode.NEAREST, transforms.functional.InterpolationMode.NEAREST], prob=0.5), 
        #         RandomFlip(keys=["img", "seg"], prob=0.5)
        #     ]
        # )

    def set_feat_model(self, model_feat):
        self.F.module = model_feat
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
        return optimizer
    
    def forward(self, X_patches):

        # x_bb = torch.stack([self.compute_bb(seg, pad=0.01) for seg in X["seg"]])
        # X_padded = torch.stack([self.pad_image_to_fixed_size(img, bb) for img, bb in zip(X["img"], x_bb)])

        # X_patches = [self.extract_patches(img_padded) for img_padded in X_padded]
        # X_patches = torch.stack(X_patches)

        x_f = self.F(X_patches)
        x_v = self.V(x_f)

        ##### Multihead Attention #####
        x_v2a = self.V2A(x_v)
        x_a, x_a_weights = self.A(x_v2a, x_v2a, x_v2a)  # Shape [BS, n_patches^2, 1024],[BS, n_patches^2, n_patches^2]

        # use the weights to update x_a
        sum_weights = torch.sum(x_a_weights, 1, keepdim=True)
        attention_weights = x_a_weights / sum_weights   # Shape: [BS, n_patches^2, n_patches^2]

        ##  mat1 (b×n×m), mat2 (b×m×p), out is (b×n×p)
        x_a = x_a.transpose(1, 2)  # Shape: [BS, 1024, n_patches^2]
        x_a = torch.bmm(x_a, attention_weights)  # Shape: [BS, 1024, n_patches^2]
        x_a = x_a.transpose(1, 2)

        x = self.P(x_a)
        return x, X_patches, x_a, x_v,

    def training_step(self, train_batch, batch_idx):

        imgs, labels = train_batch['patches'], train_batch['labels']
        # batch = self.train_transform(imgs)

        x, X_patches, x_a, x_v, = self(imgs)
        x = x.reshape(-1,self.num_classes)

        loss = self.loss(x, labels.reshape(-1))
        self.log('train_loss', loss, sync_dist=True)

        self.accuracy(torch.argmax(x, dim=1), labels.reshape(-1))
        self.log("train_acc", self.accuracy, sync_dist=True)
        return loss

    def validation_step(self, val_batch, batch_idx):

        imgs, labels = val_batch['patches'], val_batch['labels']
        # batch = self.train_transform(imgs)

        x, X_patches, x_a, x_v, = self(imgs)
        x = x.reshape(-1,self.num_classes)

        loss = self.loss(x, labels.reshape(-1))
        self.log('val_loss', loss, sync_dist=True)

        self.accuracy(torch.argmax(x, dim=1), labels.reshape(-1))
        self.log("val_acc", self.accuracy, sync_dist=True)

    def test_step(self, test_batch, batch_idx):
        imgs, labels = test_batch['patches'], test_batch['labels']
        # batch = self.train_transform(imgs)

        x, X_patches, x_a, x_v, = self(imgs)
        x = x.reshape(-1,self.num_classes)

        out = [ torch.argmax(x, dim=1), labels.reshape(-1) ]
        return out

In [25]:
train_transform = TrainTransformsFullSeg()
eval_transform = EvalTransformsFullSeg()

ttdata = TTDataModulePatch(df_train, df_val, df_test, 
                         batch_size=2, num_workers=4, 
                         img_column='filename', 
                         mount_point=mount_point, drop_last=True)


In [26]:
ttdata.setup()
dataload = ttdata.train_dataloader()
ds = ttdata.train_ds

In [9]:
for idx, batch in enumerate(tqdm(dataload)):
  patches, labels, img = batch['patches'].cuda(), batch['labels'].cuda(), batch['img'].cuda()
  
  data = {"patches": patches, "labels": labels, "img":img}
  
  break

  0%|          | 0/12207 [00:00<?, ?it/s]The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for 

In [27]:
model = EfficientNetV2SYOLTPatchv2(num_classes=7).cuda()

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.


In [32]:
model.test_step(data, 0)